### Environment Preparation 

* prepare session and bucket 
* prepare necessary parameters 

In [1]:
import sagemaker 
import boto3 
sess = sagemaker.Session()
BUCKET = sess.default_bucket()

In [2]:
BUCKET

'sagemaker-us-west-2-518460648208'

In [3]:
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]


In [4]:
!ls 


01-environment.ipynb			      deploy_model.py
02-a2i-object-detection-and-retraining.ipynb  images
03-prepare-lambda-functions.ipynb	      invoke_image_object_detection
api.sh					      process_a2i_results
augmented.manifest			      README.md
content_image				      sample-a2i-images


### Setup lambda functions and SQS 
We use a lambda function ```invoke_image_object_detection``` to 
1. trigger the sagemaker endpoint to do inference
2. save the image to s3 and trigger a2i process 
3. send a2i and s3 information to SQS 

And, we use a lambda function ```process_a2i_results``` to 
1. listening to SQS and process a2i results 
2. trigger retraining pipelines 

The following steps creates lambda functions 

* copying codes to s3 
* create lambda functions by codes on s3 
* setup environment variables 


In [5]:
%%bash -s "$BUCKET" 
cd invoke_image_object_detection 
zip -r invoke_image_object_detection.zip  .
aws s3 cp invoke_image_object_detection.zip s3://$1/lambda/
cd ../process_a2i_results
zip -r process_a2i_results.zip  .
aws s3 cp process_a2i_results.zip s3://$1/lambda/

updating: lambda_function.py (deflated 57%)
updating: pass_tasks2sqs.py (deflated 45%)
upload: ./invoke_image_object_detection.zip to s3://sagemaker-us-west-2-518460648208/lambda/invoke_image_object_detection.zip
updating: prepare_data.py (deflated 67%)
updating: model.py (deflated 47%)
updating: lambda_function.py (deflated 62%)
upload: ./process_a2i_results.zip to s3://sagemaker-us-west-2-518460648208/lambda/process_a2i_results.zip


In [6]:
!aws lambda create-function --function-name invoke_image_object_detection --zip-file fileb:///home/ec2-user/SageMaker/mlops_incremental_learning/invoke_image_object_detection/invoke_image_object_detection.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role arn:aws:iam::$account_id:role/TeamRole


{
    "FunctionName": "invoke_image_object_detection",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:invoke_image_object_detection",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::518460648208:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 1578,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-06T13:57:45.017+0000",
    "CodeSha256": "cdJXlHHH1xbUUHL+k1pSzj1WEadtRqUFBSZ1wDxOh2I=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "7d3e9752-8b3f-4a24-a312-227bc671ad1e",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


#### Configure lambda function - invoke_image_object_detection 
* you can also do it by command line - 
```
aws lambda update-function-configuration --function-name invoke_image_object_detection \
    --environment "Variables={BUCKET=my-bucket,KEY=file.txt}"
```    
![configure environment variable](./content_image/setup_env_vars_for_lambda2.png)

In [25]:
%store -r flowDefinitionArn
%store -r endpoint_name

In [26]:
sqs = boto3.resource('sqs')
queue = sqs.create_queue(QueueName='a2itasks', Attributes={'DelaySeconds': '0'})
print(queue.url)



https://us-west-2.queue.amazonaws.com/518460648208/a2itasks


In [27]:
queue

sqs.Queue(url='https://us-west-2.queue.amazonaws.com/518460648208/a2itasks')

In [34]:
bucket_key = "a2i-demo"
variables = f"A2IFLOW_DEF={flowDefinitionArn},BUCKET={BUCKET},ENDPOINT_NAME={endpoint_name}, SQS_URL={queue.url}, KEY={bucket_key}"
env = "Variables={"+variables+"}"

!aws lambda update-function-configuration --function-name invoke_image_object_detection --environment "$env"


{
    "FunctionName": "invoke_image_object_detection",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:invoke_image_object_detection",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::518460648208:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 1578,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-06T14:24:05.026+0000",
    "CodeSha256": "cdJXlHHH1xbUUHL+k1pSzj1WEadtRqUFBSZ1wDxOh2I=",
    "Version": "$LATEST",
    "Environment": {
        "Variables": {
            "ENDPOINT_NAME": "DEMO-object-detection-augmented-ai-2021-05-06-13-24-14",
            "A2IFLOW_DEF": "arn:aws:sagemaker:us-west-2:518460648208:flow-definition/fd-sagemaker-object-detection-demo-2021-05-06-13-34-10",
            "BUCKET": "sagemaker-us-west-2-518460648208",
            "KEY": "a2i-demo",
            "SQS_URL": "https://us-west-2.queue.amazonaws.com/518460648208/a2itasks"
        }
    },
    "TracingConfig":

In [29]:
!aws lambda create-function --function-name process_a2i_results --zip-file fileb:///home/ec2-user/SageMaker/mlops_incremental_learning/process_a2i_results/process_a2i_results.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role arn:aws:iam::$account_id:role/TeamRole

{
    "FunctionName": "process_a2i_results",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:process_a2i_results",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::518460648208:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 2521,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-06T14:16:48.220+0000",
    "CodeSha256": "gyO803L3VA6jZzDdwtfICS3+EGMyUAUsg/gbhjOh2zM=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "c0615abc-f972-482e-96b2-5c2f1cad5ebd",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


#### Configure lambda function - process_a2i_results
![configure environment variable](./content_image/setup_env_vars_for_lambda3.png)

In [30]:
%store -r model_package_group_name 
%store -r pipeline_name

In [31]:
bucket_key = "a2i-demo"
variables = f"MODEL_GROUP={model_package_group_name},BUCKET={BUCKET},PIPELINE={pipeline_name}"
env = "Variables={"+variables+"}"

!aws lambda update-function-configuration --function-name process_a2i_results --environment "$env"

{
    "FunctionName": "process_a2i_results",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:process_a2i_results",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::518460648208:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 2521,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-06T14:20:44.601+0000",
    "CodeSha256": "gyO803L3VA6jZzDdwtfICS3+EGMyUAUsg/gbhjOh2zM=",
    "Version": "$LATEST",
    "Environment": {
        "Variables": {
            "BUCKET": "sagemaker-us-west-2-518460648208",
            "PIPELINE": "ImageObjectDetection",
            "MODEL_GROUP": "ObjectDetectionGroupModel"
        }
    },
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "3359106f-a16c-462b-b043-4105c751666a",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


### Set up SQS 

* we use SQS to pass the information between lambda functions in this solution 
* set up SQS and attach it as a trigger to a lambda function 



In [33]:
!aws lambda create-event-source-mapping --function-name process_a2i_results  --batch-size 10 \
--maximum-batching-window-in-seconds 300 --event-source-arn arn:aws:sqs:us-west-2:$account_id:a2itasks



An error occurred (ResourceConflictException) when calling the CreateEventSourceMapping operation: An event source mapping with SQS arn (" arn:aws:sqs:us-west-2:518460648208:a2itasks ") and function (" process_a2i_results ") already exists. Please update or delete the existing mapping with UUID 2249c63f-a4c6-4622-8e2b-ef7a46f4979c


### Test Material 
```
{
  "content": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFRUZGRgYGBgYGBoYGBoYGBgYGhgZGRgYGBgcIS4lHB4rIRgYJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHxISHjQrJCQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIAMMBAgMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAAEAAIDBQYBB//EADkQAAEDAgQEBAMGBgIDAAAAAAEAAhEDIQQSMUEFBlFhEyJxgTKhsUKRwdHh8BQVUmJygiPxB5Ky/8QAGQEAAwEBAQAAAAAAAAAAAAAAAAECAwQF/8QAJBEAAgICAwACAgMBAAAAAAAAAAECESExAxJBE1EiMgRhgXH/2gAMAwEAAhEDEQA/AABjTG6aKyAZWC67EhcDbDsWLn2QtWoVC3EAqUEKbYdgd9KdUFX4c0o6u6NEmGVomIqTw0dENW4SDstA4BDuTUmBS0uDiVLU4SNCrunAXKjgm5sADDcHaRGUKj4hwlzXWG61rKkBQYhwKcZtOxUYd+FcNk0UndFr3YVpSfgGwtPlY8mNLT0SAK1zeHN6JzeGN6I+VBZksvUKMha+rwodFEzgwnRC5UKyhZmy2BQtQkm62/8ALQGqlxXB5d5QlHkVgmZ9JXL+COCHfwp4WqnFjtFckjv5a9NdgHBO0FoDSUpw7ui4aThsU7QyNJSCiei4WEbItAMBSKSUIASSRCSAEkkkgDXGuVzxVASmBy5EhBza6kOKIQTXJr3lNxFQd/FSnnGQFVhyeRZLqJhYxqe3EKtIU9JKgDhXSa8ygy6EvGRQFk+quEygGVC4wFe4HglR0F8U2/3fEfRv5otIqKlJ0kQ4PDh13TlHTUnoFd/wdF7LMLSdHAkkHvJgqQiixuUbD7h17pYbiDG7SD1tH5LGUm3g7YcUVGmsgNPgDyfLUZHW/wAwmYzg1anfLnb/AFMkx6iJC0+EOe9o7z+KPZiabCJqNHUG6PkfoPgi9HnLnwuU6q9QxWDoV2Fr8pkGHNABaToQUPgeWMMwAtYXnWXmflp8k1NUYvgdmCbRe4SGOI6wYXKZG4Xrjn5GEaNANsrdIVZR5awrpc5mYuuczjN+gBgeyblHwn4JHmtVzUK9jVveJcq4UuysL6ZO4OZs9CHfmlheWcOxozAVXaOLiW/+rZshTig+CVmDZRaVFWwjdoXoLKGHpkinTDHdYLj9529E19U7NbHVoAjuRCPmXhov4rrLPPsNwd9Qwxhd6Cw9ToEY/k6pEuLG9nOP1AIWsdiHTrPr+Sc7ENcMpspfO/DSP8WK2zB47l+pREvZDTo5pDmH/YWB7FVVTBjovTWPyTo5hEOY64c3cELMce4c2k/yXY8B7JuQD9knqD+C0hy9jDm4emVoyD+HjonNwQ6K2cJ2SFNbdznKl3D5GiFOAutHkEIOsxNTHZWfwASRmUrqfYLDGUJTH0YKJpPSqGVnYwRzEwMRLmKNzYRYEMQkSpMspgYiwIi1PYYTnNUYclQrHuN04XSaJTiIQInw9QsJe0eYAlvr+cSh8XzBVcfjOnX8FzOg8XQnzN9x36hHVXk24pUqHjiT+pU9HiD5EEkuP00CqQYN0XhjLgRtum4pG/Z/ZccR5kfTbkYb6SCbnc+nRZ3+ZVnuBL3EylisI74j6W2RXC8KHvAPwi59AqqEY3RPaUpbwekcv8ScWMzAxb1J6wt3hXktDoi2i8xw/GKVDLmgWnsBt7q84TzjTqmA6DMRtA3XG00rrB0YeLNgAXlzSYGVxI9GkoWhXd11+SJ5ZqNqB5I1zN9oVFUxDWPex5gA2J0hS1UVL7sE/wAnEvThcw819x++qz/N1Ooyk57ZMAzHyPqoqfMtNj8hqNE6EuhaV9dlRmUwQ6x/RKsZQ7aPH8BzkWuAcTGkm/4LT8O4uysYY4B/TZ3p+SxfNPAhTqPDLQ4/mFV8IbUaZBjKZB6EdCuh8UJR7QZHyyUqkj1l7JvEOF+nuFVcRxTWAuecsW/6R2D4wx1NheQX5fNpqLGR+9VVcb4hScIDG37yfumAsEldGzdK7HYfEB1Jry17swJADg2GCwJsTJIMe3VVfF6pc5l5bkGWdQNCD3kIetxBxblFhA+URHawQufSSt4xrJx83KpKkyVRl9106KGbrQ5iYlNc0LhNlHmQJnMgSTl1FsQxlRPc9KrRhOpM7JlCYDqlUKMY2yGfR1QDQM1yTinChdJ1IpEkVRQEoh7CozQKYDfEXDUXXUCuHDlNIdHJXXFMa0p5TBCYAdQnuaFxgXXBSxj2MEQpOG4bK5xm0W97/gowYRVB3lJlTLRpwvNFdj8F4jrHzDrpCs+W+WanjDzNiL7g9h30+9Op4HP5i8NHpeB6LX8Ir0KNMPfNogadSJ7mFPeTj1R0qMU+zN5ytg8gaAIDRfue6B5q4CXguYGk31EiFQYPnOS4spuIsBDgIJMCVpeX+YxiC6i9jvEa0lwdAcWyBItfULX4bhTM/lqdo8pfyjUc5xcXQTldo2zbi3QbEL0Hlrh7WMaA5xDRAzOLr+6t+NcGNsroB+1Egjo4dUM0CiweabWgRJ9FzzlJvq/DaPWrj6ed89tjEubPlLWu++fnqsy58aLU84DNUncMbJtrLjH3ELLuYtOJLqc/M25UJlSVMAhskKRj1ZiSOXIXQ5LMihCkppUkrhRQUNldyroCdMIoBmQpKXOEk6YqQhVlPY8KLw4TgEgoKZXCe9wIQDmFRlzkUCDMi5lUdOoVIXooZwNvdEeGIQbqhldNYwl1AkewJll2kZ1XKohOgGeCCoX0EQEg2UACikniknudCZnQA0005php/f73XAV10EaxEEe179kpLBUHUkF4WuAWg2uB6g6fijuNYx9TDupC0GZGttB81VcWwrm0i9mkNe0ja6bw/iQqAtLocR5mnQ9x1CcI1+RvOVqjPYZxaS3PAMAwT5TJE+0fNek/+I8C5uIdWNUuY1mT+0ucQXBpJkgR2uvOcVSptcZdfNcC8X6jZej8kcwUKNFzc7Z11yxGgJN102YHp3NXFGUcK+pJMRA1MyNFh38S8anTqmxJyR/cQTMeyp+LcXdxF4oUSW0ZzPfoXwW/1XiJ7kxoNdlyvwSS177MYZYztoCfYBcvNBykkjfjkoxyUXH+DEMNR5Azhoa37XwgG+5Xn+Jplji06g/9Fem88uL8Y0D4W0wBqbkmbfcvPeOUyHj/AB1PqUsRfVCmrj2YCxdKa0JwKujnOTCRK48pjE6AJY+y4EwBdDUBY8PSe6UxwhNzpCsV0kvEC4jsItxQldGHurQUhoFK3DBTZp1KR7OygqM7K6xVMdFBTpgo7B1KtrCphRI1CsG0bpzxKVh1K4YeV11EKwDAntoAosVFO+idkvDO6tBSE3ThRBTsKKhrF1jJVwcM0KIUgEdvA6lX4EqB1FXooKF2HCaYqKplJONIaQrNmGB0RLMFNgJhDd4KSoi4dRz0sr7QS2+mUjT0us3U4Pkqmm6A5zXZO9iT8gfvWqxOJyQ0jaD0lA8QoGs1rmOAqMByE76QAdiri+rLu2ZkcEOkbj63Wm4TwSmKbxiDkDv+Ng3L3tcWH08s+yhwPNLaYIq4QmoCIc13lJ7tIstNw59TFllSrTbSptcHBhIzPI0uR0kbfEffZtGlxStGo5Q5dpYZjAYLy3M+e9hbQDULR1MSxj8jdSJPtsqHhwqOqOc9rhmFyLgCIG+yu6GFp2MS7QTss1oybtlPx3AzUa/KYy3cBN9gs5zByz4tPMweZoJZ/deXMPrt3XqXgiIOioeKUgHQ0RfQad1hzRcfyTNYSUl1Z4ezCHQjRNfhey2vNHDPDf4jR5HmT0Dzcj3uVngJKcZWrMJR6uipOGURoFW9SneEhTA1VWTRU+GUvDKtHsErhYITsKKt7ZUIZdW3gjRdZhQCkKiq/h+xSV/4DUkBQeydVwYqLJ+HqWQ1dt1kW2OqvlDiWlTNlJ7eqdBY9olNewhcBI0Uk9U6GRN1RTBZQ09ZTnPG6SQqGVmrlJymzghQYcyYVD0Odqk9kXUlamBdNAzeil0sieBuddYQpXUQdCh3UyClGSehJ4Jm0byp8HZ+utvyQZqGLIjhzC57fWVcbspBGNwuYEbqh8CPKTBBsfpC1WIYZsEFiKDHxnEEHUReFpJDRDgwx13tBdu4tABP7hX/AA54eHUnR8JyWgAi7dFFgeDtPwmw7D9J2ROH4U5rpDmxmsTMiDe4uNxupbaKikyy4RiX5AxpfrveOy1OEpwJce/Qqnw4ywA6ZvbtrfZWmHcSeoVRFIsWGUPxDCBzZA8wuETSTqxsfRXOKlFpkxbTwYzjODD6b2ET5SW/5C4XnIIb6r07HOgx2K874jhQDmA66dVxccqbRvyxtdgCq64JTKhulVqhOosButbOehGFG6l0UtWn0TaQKOyYtkGUyngKQtukXABJyoUmkR5ikn5uySnuT2D3EBMIBUlOnm12UlKnJ00Qk0WlggZTJcOinLBOiKFMLjqZBkXQUC1RA0QzKbiVYam6Ip5WppCor6dM9FFVwpVg/wCKdihnvgmUk2NkDKDhpom5YMoh2IMABRYhx2CdhSHsEi6ieSNkwVSLFEZ5Gil2weUDsc4qSgSTB0RTQMkkKNjxeE44EkkQVG3MKfhNQh8dj9FD4l0qZh2b1Vp07GlbLx7utwi8Ng21D8J/1k/NC8NcH6wJ6q+/k5DbSJ6GxXRtD0GcNwQZ5XAjYZomehg2Vo2iwGIF/wB7rP4Z+TyukHrNj6zoiGVMzhc/eoWBsKxNZjHEQQNo2JROG4nSFhmn/FR4/DtIzbIdlNpNvdVoRo6FYOEt0TsTUAbqqrDPyugaKTHPdIt+ScpVFhGNsqOKN8xOyyHEsoYT0d9bfitjxIyDbQLG8bHkcB2+oleev2OmX6FFVc03hQvrCLJswLqNlOdFvSOR5HGuUn4qEQWNAAUL6QzRsmkhJEuGqA6rkCUjQACYwQjAMfASTfDHVdRQYLV1AsGZS4Zh1K7XJMAqdlOQob8G/oY8rrRNgoatM6dE6gYN0k3YqZ12G1umuGVhUue5hQvqzbYaqrKqkCUMzutlPVph0HfdE0y0adLqFo887FIWyM0w1wJ0TKwzOFrKxq0xv7KAggTCd+DS8IG4e8kWClZTgF0WRAe0sJNioXPLmgDRCCiBtbMCIsmsokbJ9NwY2SLSp6r9C3dF+j3kHdoLaJuNgttspGVQTA21UmJALHAawUWEcMfy/wCV2ZzrDYR8zqvQsFjmPYBoR7ELFcvYIvYSW7fhCt8Dw17I809Jm/a6h8soukdKgpLJd43CAtzWWcw9fLULdpWjq4kMovzkCAd1g34uX5mrpcrSZjVOjWcW4jDMo1gFZ6hxZwdrH5LtWqXAON7KnxVTKZCbb2SjUt4kYDswgant6rmH44+q8AAwPU/UarJsxudjmt+KQ33NgfqtfwSm1rAGiwGvU3ue65+abqkb8UVst8U2Wz1aZWO4405JA+IgBbl7paFhOZJDwxn2Jn3/AEWaWQm/xaM/icNbulQow2US9zgbiU1rXAXFlon9nKo0DvbFkOHkOhWzKQ33TBgW5rXVJjoHLbXTcRTltkY6gu0sOfZFhTsqfAPVJW2XskixUi0xNNoEoZmINxsEXVF9fZcr4fSNwpoojw+YDM4WOiY+mbuUhaWsh5k7BDGsbNE7J2BPQpF11ABE23hWAEXE2GnVNa+ZJHeEMAdrCRA1KjewsIEI6k9okxB2Ujhm9OqT+gKr+Jc4y4aaIwMloKDreVxbsiWYkmGgbITBsixFIWaTGYp4pgGBpomPZL2z9ko6vVAdLR2TDwEdQtBvdReDY9Roj2sIAdsUI6pBch6Ahw9K56zdTGBMCbGfuTsKYc4kahPADWGR5nAj71KQKrL7l+uMhbpb6K8wjg45Tvp6rz7g+LLX/rZbHD1SRLetuxSkns6lnBTc+4ktc2ju5oe70kgfQrLMqWhaXn6jm8LEaOA8CoDsfM9h/wDv5LIStLXhjK/S8p4mGBVON8036rj6ktDeikwGFNR7GAwXECTt1Psm5dsErBY8h8vGs573khgIA7ka+116GOFeHGUS3r27qy4Nw9lGk2mwQAPck3JPeSjy20dlq+FNZ2NcrWjNYp+VpHRYvGed7ndSTKv+N4rzvpg6Egn20+aochgCNd1yqLtj5HdIFpYa8qDEsdtBurxtIBgEKsbTJcRBsrZD0QmkIEld8rW21KJGHBaUIyh5ydgEqFsTaTnRGv4JPYS4ibiy7hn5Sb3UrGfaPcosLAPDd1SRXjDskpsLRZOy5p2At6phY4X13soalBpEOdMaAdUTTpwASU78E8AeOY98ZbRclJjIZLrkHXsj2vF56dIlQCsxxiPKQhO0CyDOxT5GQW6rj6paQALm8o52Roa0aaJ2IcA3L133SavYmrIW1IcCRqNFI6mbAm0yY2GyFrNI8wM2hcw1Qxc/EIgpppDqjmOZNgNxB6qYPDQDG111gmGkjsigWFhYRfcpoa/sDxFAkAt3Ejsm0BaHbI+ocryGXsIH1KHqSLRrc9ymxNHK1VobF5GnSUPSbml3TVKrTJiGz1SwRnMGjVTkYmOBGvmCcxs63Kc+kGuEtk9dkZSoZrsItr3TX0CZS1aJY8dDcHv0K9J5W4eGMOaC4w70svO+MVXCm5wjyOafuN16by2ZoMeT8QmT02W/Ck7sJSdIF504U2rhK8Nl7WF7Y1LmDMBbUkAj3Xj+AJeMoBJGkX/7XvzKrSYDgT6rEcxcuspVRXpeRr3f8jRGUO/qaNpOo6p8kE8oSk9Hn7qBGv6q85RwZfiaZIsCXH2B/RS4pmYg6iTeNI0Wo5CwZyvqOGrsrf8AEakep+ijjjchtmybog+L4o06T3jZpj1hGtCoObHODGR8JfleOxEj5hdL0QYfEvIaHmS57jJ9bk/RPfWOUEjQW/fsiuN0w0sDfhAJM9SqmpUdodtLLhSawzWT7Owo4kua0gqKnWIJFpO/Zcp0BldJhxiFG6mdBroTO6b+xaHY5xbGUz1UNFtySNpKTKLgMsiZTcNVcHunTf0CS/slHXsBEgKei5rmEdFHVuCR8M2UT3Whvb37JWCwxuXskjG0XwPKkn1GOZTJIaPsiSVM5pI8okk23v0TGvJGZrSLkO7jpKcxjpgyA3zAQpik0Rs5iGuiLE7t6dVXV6TgczB0kKw8W0kOBOi7lJhgudZ7I65GmBeLmcGvERcdlNXJJ0kDdS0cK0ODyPNJEnS3ULrKbs+YeYGTA09AihtWyZlItYHGJ1ynYIFkuu34h5vWeiMqsc5jnEXFo0tuhKbHAtY0AWtPT1T14JANZ7/EzQQALd1aNcSJLdQCnNfmIkD23jYItktGVwEuNwNh0lLDHvYJQxOd0htxA/RTPpPnQEjYnRDswuXOWzBNvVcwdUvBaTBbr39U442LQTWeAC82AOUDqYuoWGAMgOZ2oAn9hTupjKM1xJ9j1hLh77Py7yJjZV6CYBXD3ENIOt/RdoPcxttydFY4t4BY0C9iSO43QviDxcjT8IMnp1RVDA6+Fzse12pY6B0sYPrKIxvNQo0WU2vAaKTSASPhDToNzYe0qUsJzECQ5sT0jqsVx7hDzVaARGRzBmmMjiTY9QSR6ALTjlVjLvB/+SqjmEljWFrmhzgZsS6C1ojTyg3KvsJzd/EANcWkOny3u4CWuA2vIhYXhnJpMB7iWnUC3uDC1OD4cykcrGRlEAm5deZLlU31Vh2VFg+sfOyLOEgdFt+UCf4ZhdYmY/xBIb8gF589rg62oMQfxWw5e4kxwYxpBjM0xMBwixP+rrdkcLy7E8mtBVNzU5ow7i7QFpHrmH6qehiTlcbE+YtA94H4LPczcWY+gaYcDUztGQXJElaSlgHGjMYnGF5G4M6bKTxRkDRBMx3ChqUpbPwlsTFtUxtGCHToR7rktpheAhxizm6iL7nZVbKD2u1Nzp0V3iCHDPN7QNRG6GZiQ55tobH6pyBjXUjAgibn23lDeH10Fz3CmL5e63l0/ROd5hMRt6KW1QXaAKeKaXuYJLdk3MWlvc2PvuiDRIJbF5F46p9TDicua4+UpJWGy3ZiLC40C6qvKz+o/ckrt/Y8ljiWwbW1+pQ+GeSDJJnXvdJJJaJZeY6g3yCBAbooOL0gDTIES38l1JOWmUAvYIIjdR4Wzvl8kkllHz/RLQ2qLn0H1T8WwZGmL6T/AKlJJOQIFw7bjtP0CN3aerSuJKVofgqf2RsSbKPFMANhrKSSa2J6HYNsi9/N9FC+zXRbyn6pJKvCfoJpsGVhi5YPqVVlgFV9tXD6JJKXtif7FjR+A93ge3RRcwYZnggxcPsekriSqO0UN4V8KO+27s0JJLfm/Vf9BbG1qLYeYvDbyVk34t9LHUKdNxYx1RxLW2Bza/U/eupJQ2UjR4jidVlSnlqESwk6X/5ag/Afcp6dIZ3GLnKSdz5Ekkp/qUypYJeJ3cQe6Kx9qrgNMuiSSyWjN6Ez4PcKFjfID/e76lJJNAywawZXW3UlKmC15jqfkkkmg8BKHxj0/BQYqg3MDH2ep6pJJS0xouaOGblFth16JJJKCj//2Q=="
}
```


### Test Data 

* please adjust based on your environment 
```
{
  "Records": [
    {
      "messageAttributes": {
        "s3-image-path": {
          "stringValue": "s3://sagemaker-us-west-2-230755935769/a2i-results/sample-a2i-images/pexels-photo-276517.jpeg"
        },
        "a2i-task-id": {
          "stringValue": "a78d671e-217f-474b-bc12-c029097fb7bf"
        }
      }
    }
  ]
}
```



## Configure API Gateway 
* Finally, we need an API to have the service accessible
* This API accepts image POST to it and pass the image to ```invoke_image_object_detection```

```curl -X POST -H 'content-type: image/jpeg' --data-binary @$f $OD_API | jq .``` 

* we can create it by console also by aws cli 



#### Select RESTful API 


![configure environment variable](./content_image/build_api_gateway.png)



### Create Resource and Methods 

![configure environment variable](./content_image/create_resource.png)
<br>
![configure environment variable](./content_image/create_method.png)
<br>
![configure environment variable](./content_image/create_method2.png)
<br>



### Configurations for passing the binary content to backend
* Add binary media type in ```Settings``` of this API 
* Configure which attribute to extract and fill it in event in Lambda integration 

![configure environment variable](./content_image/setting.png)
![configure environment variable](./content_image/integration.png)
![configure environment variable](./content_image/setting1.png)

In [5]:
!cat api.sh 

#!/bin/bash

#OD_API='https://2lvd1jdy8l.execute-api.us-west-2.amazonaws.com/test/classify'
OD_API='$1'

for f in $(ls images/*.jpg)
do
    curl -X POST -H 'content-type: image/jpeg' --data-binary @$f $OD_API | jq .
done


In [35]:
!./api.sh "https://jwww8ados2.execute-api.us-west-2.amazonaws.com/dev/detect" 

https://jwww8ados2.execute-api.us-west-2.amazonaws.com/dev/detect
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62019  100 25418  100 36601  11978  17248  0:00:02  0:00:02 --:--:-- 29212
"{\"prediction\": [[7.0, 0.8127302527427673, 0.15721413493156433, 0.0, 0.7096214294433594, 0.9378196001052856], [11.0, 0.03780154883861542, 0.015134811401367188, 0.0031723082065582275, 0.8184412121772766, 0.914522647857666], [15.0, 0.029461195692420006, 0.002189718186855316, 0.4213590919971466, 0.23871341347694397, 1.0], [7.0, 0.02832825295627117, 0.27839821577072144, 0.12038061022758484, 0.6453735828399658, 0.7371115684509277], [4.0, 0.025991590693593025, 0.3640754818916321, 0.7003607153892517, 0.4270908832550049, 0.8048701882362366], [4.0, 0.023809822276234627, 0.1507493257522583, 0.797016441822052, 0.21675783395767212, 0.9492141604423523], [4.0, 0.022135887295007706, 0.5181925892829895

100  224k  100 21656  100  203k  19164   180k  0:00:01  0:00:01 --:--:--  199k
"{\"prediction\": [[11.0, 0.7593555450439453, 0.25308898091316223, 0.09349274635314941, 0.876511812210083, 1.0], [7.0, 0.10462726652622223, 0.2310047447681427, 0.06398728489875793, 0.8063862323760986, 0.8924018144607544], [11.0, 0.05242132768034935, 0.2569049894809723, 0.13554777204990387, 0.7041558027267456, 0.6232196092605591], [14.0, 0.029258690774440765, 0.562475323677063, 0.2392066866159439, 0.6426206827163696, 0.3356761932373047], [14.0, 0.026198815554380417, 0.5609219074249268, 0.28531813621520996, 0.6410099267959595, 0.3865755796432495], [14.0, 0.026164833456277847, 0.3652647137641907, 0.24291828274726868, 0.4409853219985962, 0.3347063958644867], [14.0, 0.02395058237016201, 0.4671342670917511, 0.1566764861345291, 0.5374300479888916, 0.22930632531642914], [14.0, 0.0234509389847517, 0.5144001841545105, 0.2860219478607178, 0.5884013772010803, 0.3766152262687683], [14.0, 0.022369707003235817, 0.508653163

100 24386  100 17902  100  6484  20002   7244 --:--:-- --:--:-- --:--:-- 27216
"{\"prediction\": [[11.0, 0.7470675110816956, 0.3436652421951294, 0.0353580117225647, 0.8432596921920776, 0.9607151746749878], [14.0, 0.11368395388126373, 0.2755255401134491, 0.10644605755805969, 0.874901294708252, 0.9219503402709961], [11.0, 0.03752739354968071, 0.46863988041877747, 0.05793190002441406, 0.867719292640686, 0.5757886171340942], [4.0, 0.026316653937101364, 0.6812691688537598, 0.740655779838562, 0.7412723302841187, 0.8932667970657349], [11.0, 0.02626371569931507, 0.5188655853271484, 0.2525223195552826, 0.8009827136993408, 0.9922491312026978], [7.0, 0.02273465506732464, 0.454323410987854, 0.031143933534622192, 0.973677396774292, 0.9033313989639282], [4.0, 0.022289996966719627, 0.6765435338020325, 0.7931880354881287, 0.7403092980384827, 0.9529018998146057], [4.0, 0.020908024162054062, 0.6832610368728638, 0.43440812826156616, 0.742714524269104, 0.561905026435852], [4.0, 0.02077595330774784, 0.6847